In [1]:
!pip install sklearn
!pip install segmentation_models
!pip install albumentations==0.4.5

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import tensorflow as tf

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [4]:
from PIL import Image

import os
import glob
import numpy as np

import skimage.transform as sk_transform
import skimage.filters as sk_filters
from skimage.measure import label, regionprops
from sklearn import model_selection

import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [5]:
from src.DataLoader import DataLoader, make_augmentation
from src.Model import unet_model

## Load data

In [33]:
base_dir = "/storage/supervisely/processed"
ims_dir = os.path.join(base_dir, "imgs")
labels_dir = os.path.join(base_dir, "labels")
img_files = sorted(glob.glob(ims_dir + "/*.*"))
mask_files = sorted(glob.glob(labels_dir + "/*.png"))
print (f"Masks count : {len(mask_files)};\nImgs count : {len(img_files)};")

Masks count : 5709;
Imgs count : 5709;


In [32]:
BATCH_SIZE = 24
dataset = list(zip(img_files, mask_files))
train_dataset, test_dataset = model_selection.train_test_split(dataset, test_size=0.2, random_state=0)
print(f"Train dataset size {len(train_dataset)}; Test dataset size {len(test_dataset)}")

Train dataset size 4567; Test dataset size 1142


In [34]:
output_size = (256, 256)

training_augmentation = make_augmentation(output_size=output_size, is_validation=False)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, augmentation_fn=training_augmentation, output_size=output_size, shuffle=True)

validation_augmentation = make_augmentation(output_size=output_size, is_validation=True)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, augmentation_fn=validation_augmentation, output_size=output_size, shuffle=False)

In [21]:
OUTPUT_CHANNELS = 1
model = unet_model(OUTPUT_CHANNELS)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [18]:
# model.load_weights("/storage/segmentation_training/models/best_model_full_unwrapped_from_scratch_lr00001_batch24.h5", by_name=True)

## Configure losses and compile a model

In [19]:
LR = 0.0001
EPOCHS = 40
# define optomizer
optim = tf.keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
# focal_loss = sm.losses.BinaryFocalLoss()
# total_loss = dice_loss + (1 * focal_loss)
total_loss = dice_loss

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [20]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('/storage/training_result/models/best_model_full_unwrapped_from_scratch_lr00001_batch24.h5', save_weights_only=True, save_best_only=True, mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.TensorBoard(log_dir="/storage/training_result/models/logs_full_unwrapped_from_scratch_lr00001_batch24")
]

In [ ]:
history = model.fit(
    x = train_data_loader, 
    steps_per_epoch=len(train_data_loader), 
    epochs=EPOCHS, 
    callbacks=callbacks, 
    validation_data=test_data_loader, 
    validation_steps=len(test_data_loader),
    use_multiprocessing=False,
    workers=1
) 

Train for 190 steps, validate for 47 steps
Epoch 1/40
190/190 [==============================] - 496s 3s/step - loss: 0.1149 - iou_score: 0.8195 - f1-score: 0.9005 - val_loss: 0.1423 - val_iou_score: 0.7774 - val_f1-score: 0.8742
Epoch 2/40
190/190 [==============================] - 481s 3s/step - loss: 0.1138 - iou_score: 0.8199 - f1-score: 0.9007 - val_loss: 0.1420 - val_iou_score: 0.7775 - val_f1-score: 0.8742
Epoch 3/40
190/190 [==============================] - 478s 3s/step - loss: 0.1128 - iou_score: 0.8202 - f1-score: 0.9009 - val_loss: 0.1423 - val_iou_score: 0.7743 - val_f1-score: 0.8722
Epoch 4/40
190/190 [==============================] - 480s 3s/step - loss: 0.1104 - iou_score: 0.8231 - f1-score: 0.9027 - val_loss: 0.1388 - val_iou_score: 0.7798 - val_f1-score: 0.8757
Epoch 5/40
190/190 [==============================] - 479s 3s/step - loss: 0.1101 - iou_score: 0.8227 - f1-score: 0.9024 - val_loss: 0.1384 - val_iou_score: 0.7801 - val_f1-score: 0.8759
Epoch 6/40
190/190 [==

# print("Done")

In [ ]:
# !mkdir /storage/final_model

In [ ]:
# model.save("/storage/final_model")

In [ ]:
model.save("/storage/final_model.h5")

In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 257, 257, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
#imgs, masks = test_data_loader[0]
imgs, masks = train_data_loader[0]

In [ ]:
batch_res = model.predict(imgs)

In [ ]:
plt.imshow(batch_res[0].squeeze())

In [ ]:
plt.hist(batch_res[0].squeeze().ravel(), bins=100)